# 0. Dependances

In [ ]:
!pip install tensorflow

In [ ]:
print("hello")

In [ ]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [ ]:
df = pd.read_csv("/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv")
df.head()

# 1. Data Preprocessing

In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
X = df['comment_text']
y = df[df.columns[2:]].values  
#.values is to convert the pandas series/dataframe into a numpy array

In [ ]:
MAX_FEATURES = 200000 #number of tokens (words) in a sentence

In [ ]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                              output_sequence_length=1800,
                              output_mode='int')

In [ ]:
vectorizer.adapt(X.values)

In [ ]:
vectorizer("Hello, my name is Abdo. Nice to meet you")[:9]

In [ ]:
vectorized_text = vectorizer(X.values)
vectorized_text

### Basic data pipeline:
### MCSHBAP : MAP, CACHE, SHUFFLE, BATCH, PREFETCH

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text,y))  
#TF dataset piepline
dataset=dataset.cache().shuffle(160000).batch(16).prefetch(8)

In [ ]:
dataset.as_numpy_iterator().next()

In [ ]:
train = dataset.take(int(0.7*len(dataset)))
val = dataset.skip(int(0.7*len(dataset))).take(int(0.2*len(dataset)))
test = dataset.skip(int(0.9*len(dataset))).take(int(0.1*len(dataset)))

In [ ]:
len(train),len(val),len(test)

# 2. Create Sequential Model

In [ ]:
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional,Dense,Embedding
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()
#Embedding layer: like a character test اختبار شخصية, outputs a vector of word specs
model.add(Embedding(MAX_FEATURES+1,32))

#Bidirectional LSTM: enables moving to left and right, 
model.add(Bidirectional(CuDNNLSTM(32))) #GPU with LSTM requires TANH

#Feature extractor fully connected layers
model.add(Dense(128,activation = 'relu'))
model.add(Dense(256,activation = 'relu'))
model.add(Dense(128,activation = 'relu'))

#Output layer
model.add(Dense(6,activation = 'sigmoid'))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, Dense, Dropout, BatchNormalization
from tensorflow.compat.v1.keras.layers import CuDNNGRU

In [ ]:

# Define the model architecture
model = Sequential()
model.add(Embedding(input_dim=MAX_FEATURES+1, output_dim=32, input_length=max_sequence_length))
model.add(Dropout(0.2))  # Add dropout for regularization
model.add(Bidirectional(CuDNNGRU(64)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(6, activation='softmax'))

# Add batch normalization layers
model.add(BatchNormalization())
model.add(BatchNormalization())
model.add(BatchNormalization())

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


In [ ]:
model.compile(loss = 'BinaryCrossentropy',optimizer='Adam',metrics='accuracy')
#BinaryCrossentropy because this is not a classification problem where output 
#must be a 1 and the rest are zeros, in this problem, a comment can be 
#severe toxix and threat at the same time 

In [ ]:
model.summary()

In [ ]:
history = model.fit(train,epochs=1,validation_data=val)

In [ ]:
history.history

In [ ]:
model.evaluate(test)

# 3. Make Predictions

In [ ]:
input_text = vectorizer("You suck!")

In [ ]:
df.columns[2:]

In [ ]:
predictions = model.predict(np.expand_dims(input_text,0))
results = {}
for i in range(6):
    results[df.columns[2:][i]] = predictions[0][i]

In [ ]:
for k in results.keys():
    if results[k]<0.5:
        results[k]=0
    else:
        results[k]=1
        
results
            

# 4. Evaluate the Model

In [ ]:
from tensorflow.keras.metrics import Precision,Recall,CategoricalAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    X_true,y_true = batch
    yhat = model.predict(X_true)
    
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true,yhat)
    re.update_state(y_true,yhat)
    acc.update_state(y_true,yhat)

In [ ]:
print(f"Precession: {pre.result().numpy()},Recall: {re.result().numpy()},Accuracy: {acc.result().numpy()}")

In [ ]:
from tensorflow.keras.models import load_model as tfk__load_model
# save fitted model to file
model.save('model.h5')